# CSV Import
Dask only import if one of the Inno Columns is ==True; 

Add all in one df (gpd?)


# Get coordinates for Tweets (Geocoding)
--> Via dict or list of locations (so we don't geocode everything)

--> Slight noise for tweets at same city?

--> Only one goecode per user?

# Visualize
--> Make a time based visualization for each Innovation